In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

from flask_cors import CORS, cross_origin
from flask import Flask, jsonify, request
from flask import Flask, render_template, request
from keras.models import load_model
from keras.optimizers import SGD


import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework import ops
import numpy as np
import pandas as pd
import pickle
import json
import random



In [2]:
data = pickle.load(open( "chappy-bot-data.pkl", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data ['train_y']

with open('intents.json') as json_data:
    intents = json.load(json_data)
# model = load_model('chappy-bot-model.h5')

In [3]:
model = keras.Sequential()
model.add(keras.layers.Dense(128, input_shape=(len (train_x[0]),), activation ='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [4]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words


def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)

    for s in sentence_words: 
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return (np.array(bag))

In [5]:
def predict_class(sentence, model):
    p = bow (sentence, words,show_details=False)
    res = model.predict(np.array([p])) [0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
        return return_list


In [6]:
p = bow("I love you", words)
print(p)
print(classes)

found in bag: i
found in bag: lov
found in bag: you
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
['Hate', 'None', 'age', 'banter', 'body-pain', 'breathing', 'cough', 'diarrhoea', 'fever', 'goodbye', 'greeting', 'headache', 'name', 'nasal', 'noanswer', 'options', 'sore-throat', 'thanks', 'tiredness', 'unknown', 'vomit', 'yes']


In [7]:
global graph
graph = ops.get_default_graph()

with open(f'chappy-bot-model.h5', 'rb') as f:
    model = keras.models.load_model('chappy-bot-model.h5')

In [8]:
#  coughQuestionAsked = False
#  feverQuestionAsked = False
#  tirednessQuestionAsked = False
#  difficultyInBreathingQuestinAsked = False
#  noneResponse = False

#  hasCough = False
#  hasFever = False
#  hasTiredness = False
#  hasDifficultyInBreathing = False
#  hasNone = False

In [9]:
# def assessmentCheck(response):

    # global coughQuestionAsked, feverQuestionAsked, tirednessQuestionAsked, difficultyInBreathingQuestinAsked, hasCough, hasFever, hasTiredness, hasDifficultyInBreathing, noneResponse, hasNone
    
    # feedback = "continue"
    # if coughQuestionAsked == False or feverQuestionAsked == False or tirednessQuestionAsked == False or difficultyInBreathingQuestinAsked == False or noneResponse == False:
    #     if response == 'cough':
    #         coughQuestionAsked = True
    #         hasCough = True
    #         feedback = "Which of these other symptoms do you have: Fever, Tiredness, Difficulty in Breathing"

    #     if response == 'Fever':
    #         feverQuestionAsked = True
    #         hasFever = True
    #         feedback = "Which of these other symptoms do you have: Cough, Tiredness, Difficulty in Breathing"
            
    #     if response == 'Tiredness':
    #         tirednessQuestionAsked == True
    #         hasTiredness = True
    #         feedback = "Which of these other symptoms do you have: Cough, Fever, Difficulty in Breathing"

    #     if response == 'Difficulty in breathing':
    #         difficultyInBreathingQuestinAsked == True
    #         hasDifficultyInBreathing = True
    #         feedback = "Which of these other symptoms do you have: Cough, Fever, Tiredness"

    #     if response == 'None':
    #         noneResponse = True
    #         hasNone = True
    #         feedback = "Great"


    # if hasCough == True and hasFever == True and hasTiredness == True and hasDifficultyInBreathing == True:
    #     feedback = "Warning!!! You might have covid. Go for test Immediately - High risk 4"
    # if hasCough == True and hasFever == True and hasTiredness == True and hasDifficultyInBreathing ==  False:
    #     feedback = "Warning!!! You might have covid. Go for test Immediately high risk 4"
    # if hasCough == True and hasFever == True and hasTiredness == False and hasDifficultyInBreathing == True:
    #     feedback = "Warning!!! You might have covid. Go for test Immediately high risk 4"
    # if hasCough == True and hasFever == False and hasTiredness == True and hasDifficultyInBreathing == True:
    #     feedback = "Warning!!! You might have covid. Go for test Immediately high risk 4"
    # if hasCough == False and hasFever == True and hasTiredness == True and hasDifficultyInBreathing == True:
    #     feedback = "Warning!!! You might have covid. Go for test Immediately high risk 4"
    # if hasCough == True and hasFever == True and hasTiredness == False and hasDifficultyInBreathing == False:
    #     feedback = "Cough and Fever are strong symptoms of the COVID-19 disease. I advise you to stay at home and self-isolate"
    # if hasCough == True and hasFever == False and hasTiredness == True and hasDifficultyInBreathing == False:
    #     feedback = "Cough and Tiredness are strong symptoms of the COVID-19 disease. I advise you to stay at home and self-isolate"
    # if hasCough == True and hasFever == False and hasTiredness == False and hasDifficultyInBreathing == True:
    #     feedback = "Cough and Difficulty in Breathing are strong symptoms of the COVID-19 disease. I advise you to stay at home and self-isolate"
    # if hasCough == False and hasFever == True and hasTiredness == True and hasDifficultyInBreathing == False:
    #     feedback = "Fever and Tiredness are strong symptoms of the COVID-19 disease. I advise you to stay at home and self-isolate"
    # if hasCough == False and hasFever == True and hasTiredness == False and hasDifficultyInBreathing == True:
    #     feedback = "Fever and Difficulty in Breathing are strong symptoms of the COVID-19 disease. I advise you to stay at home and self-isolate"
    # if hasCough == False and hasFever == False and hasTiredness == True and hasDifficultyInBreathing == True:
    #     feedback = "Tiredness and Difficulty in Breathing are strong symptoms of the COVID-19 disease. I advise you to stay at home and self-isolate"
    
    # if hasCough == True and hasNone == True:
    #     feedback = "Sometimes Coughs could be a little bother. Wear your Face Mask while in the college at all times and see a Pharmacist."
    # if hasFever == True and hasNone == True:
    #     feedback = "Sorry about the Fever. I advise you stay at home and call a Doctor."
    # if hasTiredness == True and hasNone == True:
    #     feedback ="Sometimes I get binary tired too. I advise you stay at home and call a Doctor."
    # if hasDifficultyInBreathing == True and hasNone == True:
    #     feedback = "Sometimes I also find it hard to breath with all these algorithm processing. But, I advise you wear a Face Mask while in the college at all times and see a Pharmacist."

    # return feedback


In [10]:


def getResponse(ints, intents_json):
    responseArray = []
    tag = ints[0] ['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if (i ['tag'] == tag):
            responseArray.append(tag);
            print(i['tag'])
            print(tag)
            
            result = random.choice(i['responses'])
            responseArray.append(result);

            # feed = "continue"
            # if i['tag']== "cough" or i['tag']== "Fever" or i['tag']=="Tiredness" or i['tag']=="Difficulty in breathing" or i['tag']=="None":
            #     feed = assessmentCheck(i['tag'])

            # if feed != "continue":
            #     result = feed
           
            print (result)
            break
    
    
    
    return responseArray



def chatbot_response(text):
    ints = predict_class(text, model)
    print(ints)
    res = getResponse (ints, intents)
    return res

def chatbot_prediction(text):

    predict_list = text.split(',')
    predict_test = [predict_list]
    loaded_model = pickle.load(open('knnpickle_file', 'rb'))
    result = loaded_model.predict(predict_test) 
    print(result)

    return result




In [11]:
predict_class('I hate you', model)

[{'intent': 'Hate', 'probability': '1.0'}]

In [12]:
predict_class('Hello', model)

[{'intent': 'greeting', 'probability': '1.0'}]

In [13]:
predict_class('I have a cough', model)

[{'intent': 'cough', 'probability': '1.0'}]

In [14]:
predict_class('Yes, I have a cough', model)

[{'intent': 'cough', 'probability': '1.0'}]

In [15]:
predict_class('now', model)

[{'intent': 'unknown', 'probability': '0.60411906'}]

In [16]:
app = Flask(__name__)
app.static_folder = 'static'

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
# def classify():
    
    userText = request.args.get('msg')
    return jsonify(chatbot_response(userText))
    # return str()
    # return str("Hello")

@app.route("/post",methods = ['POST'])
def post_bot_response():
# def classify():
    
    userText = request.form.get('msg')
    return str(chatbot_prediction(userText)[0])   

if __name__ == "__main__":
    app.run() 

* Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Sep/2020 11:56:52] "GET /static/styles/style.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 11:56:59] "GET /get?msg=Hello HTTP/1.1" 200 -
[{'intent': 'greeting', 'probability': '1.0'}]
greeting
greeting
Hi there, how can I help?
[2020-09-09 11:57:03,167] ERROR in app: Exception on /get [GET]
Traceback (most recent call last):
  File "C:\Users\Detail\anaconda3\envs\chat_bot\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Detail\anaconda3\envs\chat_bot\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Detail\anaconda3\envs\chat_bot\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  F